In [33]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import glob
import os

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.float_format ='{:,.3f}'.format

In [34]:
current_date = dt.date.today()
day = current_date.day - 1 # данные по продажам мы получаем за предыдущий день
month = current_date.month
year = current_date.year

In [35]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

df_dwh = pd.read_excel(latest_file)

In [36]:
print(latest_file)

\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\2024-06-01_2024-06-02.xlsx


In [37]:
# мы смотрим продажи со времени последнего мониторинга, для этого определяем день недели и в зависимости от этого получаем новые данные
data = dt.datetime.isoweekday(dt.date.today())
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
df_dwh = df_dwh[df_dwh['Дата'] >= f'{last_monitoring}']

In [38]:
df_dwh.sample(5)

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
39354,4-01-152,2024-06-02 10:04:00,Елькина Екатерина Сергеевна,1011,Ch2-000847584,ТОВ1305840,КП 1,1,Футляр,"9,000,212,793,937.000",NaN,ФУТЛЯРЫ,ФУТЛЯР,0,NaN,0.000,NaN,0.000,NaN,0.000,0.000,0.000,КОРОБОЧКА,0.000,(904)380-75-72,NaN,0.000,0.000,0.000
7636,3-04-026,2024-06-01 08:19:00,Бойко Ирина Сергеевна,2347,Ch2-000843609,ТОВ1172935,CR_BOX,1,Футляр,"9,000,158,580,820.000",NaN,ФУТЛЯРЫ,ФУТЛЯР,0,NaN,0.000,NaN,0.000,NaN,0.000,0.000,0.000,КОРОБОЧКА 585*Золото,0.000,+79134383897,NaN,0.000,0.000,0.000
23135,6-05-020,2024-06-01 14:13:00,Романова Ольга Сергеевна,2837,Ch4-000531422,ТОВ1194920,35-0727-11-61,1,Товар,"7,651,019,260,072.000",БК ПОДВЕС КУЛЬТ,БК,ПОДВЕС КУЛЬТОВЫЙ,0,ИФ.БК_СР,1.964,585.000,0.000,"2,078,537,398,219.000","29,626.000",25.000,"11,150.000",КРЕСТ,"11,150.000",+79383466778,"5,306.810","11,150.000",0.000,"11,150.000"
39726,2-09-154,2024-06-02 10:13:00,Батищева Анна Юрьевна,5222,Ch3-000794984,ТОВ1279317,15.37-50с,1,Товар,"9,000,156,314,465.000",СИ СЕРЬГИ РАЗН,СИ,СЕРЬГИ-КОНГО,0,СИ_СЕРЬГИ_СРЕДН,2.909,925.000,0.000,"2,078,609,922,089.000","3,314.000",40.000,"1,328.000","СЕРЬГИ-КОНГО С А/Г,","1,328.000",+79067330433,619.990,"1,328.000",272.000,"1,600.000"
26701,6-04-023,2024-06-01 17:14:00,Григорян Заруи Татуловна,4225,Ch4-000531790,ТОВ1163703,ANTIBACTERIAL_WIPE,1,Аксессуар футляр,"4,410,024,570,230.000",NaN,ДФРС,ЧС,0,NaN,0.000,NaN,0.000,NaN,10.000,0.000,10.000,САЛФЕТКА АНТИБАКТЕРИ,10.000,+79183169737,NaN,10.000,0.000,10.000


In [39]:
df_dwh['Товарная группа'].str.contains('ЦБ|БК').value_counts()

Товарная группа
False    25061
True     10286
Name: count, dtype: int64

In [40]:
df_dwh = df_dwh[(df_dwh['Товарное направление'].isin(['БК', 'ЦБ'])) & (df_dwh['Проба'] == 585.0)]

In [41]:
df_dwh.head()

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
7,3-09-042,2024-06-01 00:09:00,Крюкова Лалита Михайловна,3333,Ch2-000841987,ТОВ01262494,431-01-0080-30200,1,Товар,"9,000,212,647,704.000",ЦБ БРАСЛЕТЫ,ЦБ,БРАСЛЕТ,0,ЦБ_ПУСТОТЕЛЫЕ >3 ГРАММ,3.699,585.000,19.000,"2,078,607,116,334.000","61,320.000",11.000,0.000,БРАСЛЕТ НОННА ПУСТ С,"22,659.000",(962)501-21-73,"14,142.610","22,659.000","4,641.000","27,300.000"
15,3-09-030,2024-06-01 00:20:00,Рудинская Юлия Андреевна,4056,Ch2-000841989,ТОВ1111769,41-01-0050-30176,1,Товар,"9,000,075,257,614.000",ЦБ БРАСЛЕТЫ,ЦБ,БРАСЛЕТ,0,"ЦБ_ПУСТОТЕЛЫЕ <=1,5 ГРАММ",1.288,585.000,17.000,"2,078,485,371,906.000","24,809.000",41.000,0.000,БРАСЛЕТ РОМБ ДВОЙНОЙ,"6,100.500",+79962635549,"2,379.790","6,100.500","1,249.500","7,350.000"
22,3-09-049,2024-06-01 00:43:00,Вербицкая Лилия Игоревна,3960,Ch2-000842011,ТОВ0880614,12.15_10,1,Товар,"9,000,212,647,995.000",БК СЕРЬГИ,БК,СЕРЬГИ-КОНГО,0,ИФ.БК_ДЕШ,0.530,585.000,1.000,"2,078,610,412,121.000","11,472.000",35.000,0.000,СЕРЬГИ-КОНГО,"4,980.000",(994)104-97-51,"1,696.310","4,980.000","1,020.000","6,000.000"
29,3-09-019,2024-06-01 00:46:00,Лебединская Екатерина Сергеевна,3643,Ch2-000841990,ТОВ1172647,51-33-0000-28661,1,Товар,"7,670,004,954,505.000",БК ПОДВЕС КУЛЬТ,БК,ПОДВЕС КУЛЬТОВЫЙ,0,ИФ.БК_ДЕШ,0.696,585.000,0.000,"2,078,607,595,846.000","16,543.000",29.000,0.000,КРЕСТ,"7,055.000",+79144805050,"2,461.610","7,055.000","1,445.000","8,500.000"
46,3-09-052,2024-06-01 01:09:00,Носкова Василиса Андреевна,1759,Ch2-000842002,ТОВ1163406,41-01-0060-30176,1,Товар,"5,851,010,186,575.000",ЦБ БРАСЛЕТЫ,ЦБ,БРАСЛЕТ,0,ЦБ_ПУСТОТЕЛЫЕ <=3 ГРАММ,2.298,585.000,19.000,"2,078,531,363,596.000","39,085.000",15.000,0.000,БРАСЛЕТ РОМБ ДВОЙНОЙ,"13,819.500",(924)153-71-41,"6,128.840","13,819.500","2,830.500","16,650.000"


In [42]:
df_dwh['Товарное направление'].unique()

array(['ЦБ', 'БК'], dtype=object)

In [43]:
df_dwh['Название'].isna().sum()

0

In [44]:
df_dwh['Название'] = df_dwh['Название'].fillna('золотое изделие')

In [45]:
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('каучук')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шнурок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('текстиль')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('хлопок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шелк')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('на нитке')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('кожа')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('жемчуг')))]

In [46]:
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'], format='%Y-%m-%d')
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [47]:
filtr_date = df_dwh['Дата'].max()

In [48]:
df_dwh['Дата'].min()

Timestamp('2024-06-01 00:00:00')

In [49]:
df_dwh = df_dwh.query("`Дата` <= @filtr_date")

In [50]:
df_dwh['Дата'].unique()

<DatetimeArray>
['2024-06-01 00:00:00', '2024-06-02 00:00:00']
Length: 2, dtype: datetime64[ns]

In [51]:
df_dwh['Товарное направление'].value_counts()

Товарное направление
БК    3775
ЦБ    2812
Name: count, dtype: int64

In [52]:
df_dwh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6587 entries, 7 to 52422
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   КБК                   6587 non-null   object        
 1   Дата                  6587 non-null   datetime64[ns]
 2   Продавец              6587 non-null   object        
 3   Номер чека            6587 non-null   int64         
 4   ID продажи            6587 non-null   object        
 5   Тов                   6587 non-null   object        
 6   Артикул поставщика    6587 non-null   object        
 7   Количество            6587 non-null   int64         
 8   Номенклатура          6587 non-null   object        
 9   Номер карты           6369 non-null   float64       
 10  Товарная группа       6587 non-null   object        
 11  Товарное направление  6587 non-null   object        
 12  Тип 1                 6587 non-null   object        
 13  Списание СберСпасибо  

In [53]:
tn = df_dwh.groupby(by = 'Товарная группа', as_index= False)[['Себестоимость', 'Вес', 'Общая сумма']].agg('sum')

In [54]:
tn['Наценка продажи оффлайн'] =  tn['Общая сумма'] / tn['Себестоимость'] - 1

In [55]:
tn = tn[~tn['Товарная группа'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО|ОПТ ОБРУЧИ')]

In [56]:
tn['price_per_gramm'] = ((tn['Общая сумма'] / tn['Вес']).round(0))

In [57]:
tn['price_per_gramm'] = tn['price_per_gramm'].replace(',', '')
tn['price_per_gramm'] = tn['price_per_gramm'].astype('int')

In [58]:
tn

,Товарная группа,Себестоимость,Вес,Общая сумма,Наценка продажи оффлайн,price_per_gramm
0,БК КОЛЬЦА,"1,308,177.100",469.666,"3,056,950.000",1.337,6509
1,БК КОЛЬЦА ОБРУЧ,"11,635,790.080","3,985.336","24,761,769.510",1.128,6213
3,БК ПОДВЕС ДЕКОР,"972,405.800",312.908,"2,173,434.530",1.235,6946
4,БК ПОДВЕС КУЛЬТ,"3,251,100.430","1,044.551","7,404,824.000",1.278,7089
5,БК СЕРЬГИ,"3,212,335.810","1,035.866","7,190,266.290",1.238,6941
9,ЦБ БРАСЛЕТЫ,"11,547,721.650","3,527.227","23,405,544.700",1.027,6636
10,ЦБ ЦЕПИ,"21,462,854.760","6,591.130","44,539,015.040",1.075,6757


In [59]:
print('Средняя цена продажи оффлайн: ', (tn['Общая сумма'].sum() / tn['Вес'].sum()).round(0))

Средняя цена продажи оффлайн:  6633.0


In [60]:
tn.columns = ['tg', 'cost', 'weight', 'price', 'Наценка продажи оффлайн', 'price_per_gramm']

In [61]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sales_offline\offline_sales_{date_now}.xlsx', index = False)

In [62]:
tn = tn.drop(columns=['cost', 'weight', 'price', 'Наценка продажи оффлайн'])
tn.columns = ['tg', '585*Золотой оффлайн продажи']

In [63]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)